### Groupby Aggregation

This notebook will explore the use of groupby in aggregations.

In [1]:
import polars as pl 

data = pl.read_csv("data/employees.csv")
print(data.head())

shape: (5, 6)
┌─────┬────────┬──────┬─────────────┬────────┬────────────┐
│ id  ┆ name   ┆ age  ┆ department  ┆ salary ┆ join_date  │
│ --- ┆ ---    ┆ ---  ┆ ---         ┆ ---    ┆ ---        │
│ i64 ┆ str    ┆ f64  ┆ str         ┆ i64    ┆ str        │
╞═════╪════════╪══════╪═════════════╪════════╪════════════╡
│ 1   ┆ User_1 ┆ null ┆ null        ┆ 123355 ┆ 2020-01-01 │
│ 2   ┆ User_2 ┆ 28.0 ┆ Sales       ┆ 118399 ┆ 2020-01-02 │
│ 3   ┆ User_3 ┆ 29.0 ┆ Sales       ┆ 88727  ┆ 2020-01-03 │
│ 4   ┆ User_4 ┆ 48.0 ┆ Engineering ┆ 71572  ┆ 2020-01-04 │
│ 5   ┆ User_5 ┆ 22.0 ┆ Engineering ┆ 81849  ┆ 2020-01-05 │
└─────┴────────┴──────┴─────────────┴────────┴────────────┘


### Simple Groupby 

The following example, will implement a simple groupby of department by salary to return the mean for each department

In [2]:
dept_salary_avgs = data.group_by("department").agg( pl.col("salary").mean() )
print(dept_salary_avgs)

shape: (5, 2)
┌─────────────┬───────────────┐
│ department  ┆ salary        │
│ ---         ┆ ---           │
│ str         ┆ f64           │
╞═════════════╪═══════════════╡
│ null        ┆ 83998.142857  │
│ Engineering ┆ 100800.96     │
│ Sales       ┆ 89187.166667  │
│ Marketing   ┆ 97095.956522  │
│ HR          ┆ 107561.619048 │
└─────────────┴───────────────┘


### Multiple Aggregation

One way to perform multiple aggregations is to define a set of expressions that can be passed on to a groupby function. The example below demonstrates this implementation.

In [3]:
summary_expr = [
    pl.col("salary").mean().alias("avg_salary"),
    pl.col("salary").median().alias("median_salary"),
    pl.col("salary").std().alias("std_salary"),
    pl.col("age").mean().alias("avg_age"),
    pl.len().alias("employee_count")
]

summary_data = data.group_by("department").agg(summary_expr)
print(summary_data)


shape: (5, 6)
┌─────────────┬───────────────┬───────────────┬──────────────┬───────────┬────────────────┐
│ department  ┆ avg_salary    ┆ median_salary ┆ std_salary   ┆ avg_age   ┆ employee_count │
│ ---         ┆ ---           ┆ ---           ┆ ---          ┆ ---       ┆ ---            │
│ str         ┆ f64           ┆ f64           ┆ f64          ┆ f64       ┆ u32            │
╞═════════════╪═══════════════╪═══════════════╪══════════════╪═══════════╪════════════════╡
│ Engineering ┆ 100800.96     ┆ 95432.0       ┆ 24746.970967 ┆ 41.76     ┆ 25             │
│ HR          ┆ 107561.619048 ┆ 106714.0      ┆ 30100.811834 ┆ 42.647059 ┆ 21             │
│ Sales       ┆ 89187.166667  ┆ 89576.0       ┆ 23739.912497 ┆ 37.391304 ┆ 24             │
│ Marketing   ┆ 97095.956522  ┆ 96023.0       ┆ 27680.241155 ┆ 37.454545 ┆ 23             │
│ null        ┆ 83998.142857  ┆ 77334.0       ┆ 29473.752173 ┆ 33.333333 ┆ 7              │
└─────────────┴───────────────┴───────────────┴──────────────┴────

#### Groupby Multiple Keys

An addditional feature that is useful is the ability to group data by multiple keys. Let's first create a new column based on age. Simply evaluate the ages, everything above media is old and below is younger.

In [4]:
data = data.with_columns( 
        pl.when(pl.col("age") >= pl.col("age").median())
        .then(pl.lit("old"))
        .otherwise(pl.lit("young")).alias("age_group"))

print(data.head())

shape: (5, 7)
┌─────┬────────┬──────┬─────────────┬────────┬────────────┬───────────┐
│ id  ┆ name   ┆ age  ┆ department  ┆ salary ┆ join_date  ┆ age_group │
│ --- ┆ ---    ┆ ---  ┆ ---         ┆ ---    ┆ ---        ┆ ---       │
│ i64 ┆ str    ┆ f64  ┆ str         ┆ i64    ┆ str        ┆ str       │
╞═════╪════════╪══════╪═════════════╪════════╪════════════╪═══════════╡
│ 1   ┆ User_1 ┆ null ┆ null        ┆ 123355 ┆ 2020-01-01 ┆ young     │
│ 2   ┆ User_2 ┆ 28.0 ┆ Sales       ┆ 118399 ┆ 2020-01-02 ┆ young     │
│ 3   ┆ User_3 ┆ 29.0 ┆ Sales       ┆ 88727  ┆ 2020-01-03 ┆ young     │
│ 4   ┆ User_4 ┆ 48.0 ┆ Engineering ┆ 71572  ┆ 2020-01-04 ┆ old       │
│ 5   ┆ User_5 ┆ 22.0 ┆ Engineering ┆ 81849  ┆ 2020-01-05 ┆ young     │
└─────┴────────┴──────┴─────────────┴────────┴────────────┴───────────┘


In [5]:
summary_data = data.filter(~pl.col("department").is_null()).group_by("department", "age_group", ).agg(summary_expr)
print(summary_data)


shape: (8, 7)
┌─────────────┬───────────┬───────────────┬──────────────┬──────────────┬───────────┬──────────────┐
│ department  ┆ age_group ┆ avg_salary    ┆ median_salar ┆ std_salary   ┆ avg_age   ┆ employee_cou │
│ ---         ┆ ---       ┆ ---           ┆ y            ┆ ---          ┆ ---       ┆ nt           │
│ str         ┆ str       ┆ f64           ┆ ---          ┆ f64          ┆ f64       ┆ ---          │
│             ┆           ┆               ┆ f64          ┆              ┆           ┆ u32          │
╞═════════════╪═══════════╪═══════════════╪══════════════╪══════════════╪═══════════╪══════════════╡
│ Marketing   ┆ young     ┆ 101336.615385 ┆ 96023.0      ┆ 29783.924309 ┆ 25.5      ┆ 13           │
│ Engineering ┆ old       ┆ 95780.142857  ┆ 89367.0      ┆ 23926.099039 ┆ 49.857143 ┆ 14           │
│ Marketing   ┆ old       ┆ 91583.1       ┆ 98821.5      ┆ 25107.819572 ┆ 51.8      ┆ 10           │
│ Sales       ┆ old       ┆ 84975.090909  ┆ 91618.0      ┆ 25500.279879 ┆ 48.